In [1]:
import numpy as np
import os
import glob
import re
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pydot
from random import shuffle
from skimage.util.shape import view_as_blocks
from skimage import io, transform
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.utils.vis_utils import plot_model
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D

Using TensorFlow backend.


In [2]:
train_size = 10000
test_size = 3000

train = glob.glob("train/*.jpeg")
test = glob.glob("test/*.jpeg")

shuffle(train)
shuffle(test)

train = train[:train_size]
test = test[:test_size]

piece_symbols = 'prbnkqPRBNKQ'

In [3]:
def fen_from_filename(filename):
    base = os.path.basename(filename)
    return os.path.splitext(base)[0]

In [53]:
def process_image(img):
    downsample_size = 200
    square_size = int(downsample_size/8)
    img_read = io.imread(img)
    img_read = transform.resize(
      img_read, (downsample_size, downsample_size), mode='constant')
    tiles = view_as_blocks(img_read, block_shape=(square_size, square_size, 3))
    tiles = tiles.squeeze(axis=2)
    return tiles.reshape(64, square_size*square_size*3)

In [29]:
tiles = process_image(train[1])
x = tiles.flatten()
x.shape

(120000,)

In [5]:
def onehot_from_fen(fen):
    eye = np.eye(13)
    output = np.empty((0, 13))
    fen = re.sub('[-]', '', fen)

    for char in fen:
        if(char in '12345678'):
            output = np.append(
              output, np.tile(eye[12], (int(char), 1)), axis=0)
        else:
            idx = piece_symbols.index(char)
            output = np.append(output, eye[idx].reshape((1, 13)), axis=0)

    return output

In [6]:
def fen_from_onehot(one_hot):
    output = ''
    for j in range(8):
        for i in range(8):
            if(one_hot[j][i] == 12):
                output += ' '
            else:
                output += piece_symbols[one_hot[j][i]]
        if(j != 7):
            output += '-'

    for i in range(8, 0, -1):
        output = output.replace(' ' * i, str(i))

    return output

In [54]:
def train_gen(features, labels, batch_size):
    for i, img in enumerate(features):
        y = onehot_from_fen(fen_from_filename(img))
        x = process_image(img)
        yield x, y

In [55]:
def pred_gen(features, batch_size):
    for i, img in enumerate(features):
        x = process_image(img)
        yield x.flatten()

In [56]:
model = Sequential()
model.add(Dense(2048, input_dim=1875,kernel_initializer='he_normal'))
model.add(Activation('relu'))
model.add(Dense(1024, kernel_initializer='he_normal'))
model.add(Activation('relu'))
model.add(Dense(13, kernel_initializer='he_normal'))
model.add(Activation('softmax'))

In [57]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit_generator(train_gen(train, None, 64), steps_per_epoch=train_size)

Epoch 1/1
